In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
from tqdm import tqdm

In [ ]:
def get_topic_url(soup):
    topic_urls = []
    for i, title in enumerate(soup.find_all('div',{'class':'category--header'})):
        icon = True if title.find('i') else False
        if not icon:
            if int(title.find('span',{'class':'topic_number'}).text) > 5:
                topic_urls.append(
                    {
                    'title':title.find('a').text.strip(),
                    'url':title.find('a')['href'].strip()
                }
                )
    return topic_urls

In [ ]:
with open('topics/shop-cloths.jsonl', 'a', encoding="utf-8") as outfile:
    for n in tqdm(range(1+681+405, 1155)):
        category_topics_url_string = 'https://www.ninisite.com/discussion/forum/89/%d9%be%d9%88%d8%b4%d8%a7%da%a9'
        category_topics_url = urlopen(category_topics_url_string+f'?page={n}')
        category_topics = category_topics_url.read()
        category_topics_soup = BeautifulSoup(category_topics, 'html.parser')
        topic_urls = get_topic_url(category_topics_soup)
        for url in topic_urls:
            json.dump(url, outfile, ensure_ascii=False)
            outfile.write('\n')

In [ ]:
with open('topics/shop-toys.jsonl') as json_file:
    data = json_file.readlines()
    data = [d.strip() for d in data]

In [ ]:
topic_url_string ='https://ninisite.com/'+eval(data[0])['url']
topic_url = urlopen(topic_url_string+'?page=1')
topic_content = topic_url.read()
topic_soup = BeautifulSoup(topic_content, 'html.parser')
pagination = topic_soup.find_all('a',{'class':'page-link'}) 
pages = 1 if len(pagination) == 0 else int(pagination[-2].text)

In [ ]:
conv = []
for p in tqdm(range(1,pages+1)):
    if pages > 1:
        topic_url = urlopen(topic_url_string+f'?page={p}')
        topic_content = topic_url.read()
        topic_soup = BeautifulSoup(topic_content, 'html.parser')

    for i, article in enumerate(topic_soup.find_all('article')):
        message = [p.text for p in article.find('div',{'class':'post-message'}).find_all('p')]

        reply_message_div = article.find('div',{'class':'reply-message'})
        reply_message_text = reply_message_div.text if reply_message_div else None
        
        if reply_message_text != None:
            if len(reply_message_text)  < 90:
                reply_message = reply_message_text
            else:
                reply_message_id = reply_message_div['data-id'] if reply_message_div else None
                
                if reply_message_id != None:
                    reply_message = topic_soup.find('article', attrs={'id':f'post-{reply_message_id}'})
                    
                    if reply_message != None:
                        reply_message = [p.text for p in reply_message.find('div',{'class':'post-message'}).find_all('p')]
                    else:
                        reply_message_url_string = topic_url_string+f'?postId={reply_message_id}'
                        reply_message_url = urlopen(reply_message_url_string)
                        reply_message_content = reply_message_url.read()
                        reply_message_soup = BeautifulSoup(reply_message_content, 'html.parser')
                        reply_message = reply_message_soup.find('article', attrs={'id':f'post-{reply_message_id}'})
                        reply_message = [p.text for p in reply_message.find('div',{'class':'post-message'}).find_all('p')]
        else:
            reply_message = topic_soup.find('article', attrs={'id':'topic'})
            reply_message_title = topic_soup.find('article', attrs={'id':'topic'}).find('h1',{'class':'topic-title'}).find('a').text
            reply_message = [p.text for p in reply_message.find('div',{'class':'post-message'}).find_all('p')]

        if i != 0:
            conv.append(
                {
                    "topic": reply_message_title,
                    "question":reply_message,
                    "answer":message,
                }
            )

In [ ]:
import os
categories_url_path = os.listdir('topics')

all_topics = []
for cat in categories_url_path:
    with open(f'topics/{cat}') as jsonl:
        topics = jsonl.readlines()
        for d in topics:
            all_topics.append(d.strip())


In [ ]:
from os.path import isfile, join
from os import listdir

def get_urls(path):
    categories_url_path =  [f for f in listdir(path) if isfile(join(path, f))]
    all_topics = []
    for cat in categories_url_path:
        with open(f'topics/{cat}') as jsonl:
            topics = jsonl.readlines()
            for d in topics:
                all_topics.append((cat, d.strip()))
    return list(set(all_topics))

def get_crawled(path):
    with open(path) as jsonl:
        craweld_topics = jsonl.readlines()
        craweld_topics = [t.strip() for t in craweld_topics]
    return craweld_topics

In [ ]:
all_topics = get_urls('topics')
craweld_topics = get_crawled('crawled_topics.jsonl')


In [ ]:
craweld_topics

In [ ]:
all_topics

In [ ]:
test = 'تنهاس تنهایی بد دردی چقد خانواده شوهرم نیش بهم زدن چقد غربت سخته .بچم با سهل انگار دکترم مرد .......'
len(test)